In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import requests
import time
from google.colab import drive
drive.mount('/content/drive')



"""
Part II: Dataset Recollection and Cleaning.

"""

AQIdf= pd.read_csv("/content/drive/My Drive/ISYE 3030 Project/annual_aqi_by_cbsa_2024.csv") #dataset containing Air Quality Data for different CBSA's accross the country
cbsa_codes = AQIdf["CBSA Code"].astype(str).unique().tolist()
#API Quality of Life Indicators Dataset API.
base_url = "https://api.census.gov/data/2023/acs/acs5"
api_key = "c6886343eae9f8942d04ffe9d2872f47f29bbdcd"
variables = [ "B19013_001E",  # Median Household Income
              "B19083_001E",  # Income Inequality (Gini)
              "B17001_002E",  # Number of People in Poverty
              "B23025_005E",  # Number of Unemployed People
              "B15003_017E",  # Number of People with Educational Attainament(HS+)
              "B25077_001E",  # Housing Affordability (Median Home Value)
              "B27010_001E",  # Number of People who have Access to Healthcare (Health Insurance Coverage
              "B28002_001E",  # Number of People with Internet Access
              "B01003_001E"   #Final Total Population
            ]

all_results = []
for code in cbsa_codes:
    params = {
        "get": "NAME," + ",".join(variables),
        "for": f"metropolitan statistical area/micropolitan statistical area:{code}",
        "key": api_key
    }
    resp = requests.get(base_url, params=params)

    if resp.status_code == 200:
        data = resp.json()
        cols = data[0]
        rows = data[1:]
        df = pd.DataFrame(rows, columns=cols)
        for var in variables:
            df[var] = pd.to_numeric(df[var], errors='coerce')
        df["CBSA_Code"] = code
        all_results.append(df)
    elif resp.status_code == 204:
        print(f"No data for CBSA {code} (204 No Content) — skipping.")
    else:
        print(f"Error {resp.status_code} for CBSA {code}: {resp.text}")

    time.sleep(1)  # avoid rate limits
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
########################################################












Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No data for CBSA 11780 (204 No Content) — skipping.
No data for CBSA 13620 (204 No Content) — skipping.
No data for CBSA 17200 (204 No Content) — skipping.
No data for CBSA 17460 (204 No Content) — skipping.
No data for CBSA 19380 (204 No Content) — skipping.
No data for CBSA 22280 (204 No Content) — skipping.
No data for CBSA 24060 (204 No Content) — skipping.
No data for CBSA 25660 (204 No Content) — skipping.
No data for CBSA 31460 (204 No Content) — skipping.
No data for CBSA 32220 (204 No Content) — skipping.
No data for CBSA 35260 (204 No Content) — skipping.
No data for CBSA 37080 (204 No Content) — skipping.
No data for CBSA 39140 (204 No Content) — skipping.
No data for CBSA 39260 (204 No Content) — skipping.
No data for CBSA 40500 (204 No Content) — skipping.


Cleaning the DataFrames we have been working with.


In [4]:


finalaqi = pd.read_csv("/content/drive/My Drive/ISYE 3030 Project/annual_aqi_by_cbsa_2024.csv")
listofCBSA = final_df["CBSA_Code"].astype(str).tolist()
#Cleaning the Quality of Air dataset also saving enviroment
finalaqi["CBSA Code"] = finalaqi["CBSA Code"].astype(str)
finalaqi = finalaqi[finalaqi["CBSA Code"].isin(listofCBSA)]
output_pathAQI = '/content/drive/My Drive/ISYE 3030 Project/cleaned_AQI.csv'
finalaqi.to_csv(output_pathAQI, index=False)

#Cleaning and Renaming the indicators in the quality of life dataset
nameDict = {"NAME":"Core Base Stastical Area","B19013_001E": "Median Household Income", "B19083_001E":"Income Inequality (Gini)", "B17001_002E":"Number of People in Poverty",
            "B23025_005E":  "Number of Unemployed People",  "B15003_017E": "Number of People With Educational Attainment (HS+)",
            "B25077_001E":"Housing Affordability (Median Home Value)",  "B27010_001E": "Number of People with Health Insurance Coverage",
            "B28002_001E": "Number of People With Internet Access", "B01003_001E": "Final Total Population"}
final_df = final_df.rename(columns=nameDict)
final_df.replace(-99999,0, inplace=True)
final_df.replace("", 0, inplace= True)
final_df.replace("N/A",0, inplace=True)
output_path = '/content/drive/My Drive/ISYE 3030 Project/cleaned_indicators.csv'
final_df.to_csv(output_path, index=False)
finaldf = pd.merge(finalaqi, final_df, left_on="CBSA Code", right_on="CBSA_Code")
finaldf.to_csv("/content/drive/My Drive/ISYE 3030 Project/FinalDataset.csv", index=False)




